In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from models import Cnet
from datasets import WSSBDatasetTest
from utils import od2rgb, rgb2od, random_ruifrok_matrix_variation, direct_deconvolution

In [ ]:
GENERATE_IMAGES = True
RUNNING_ON_DELFOS = True

torch.manual_seed(0)
plt.rcParams['font.size'] = 14
plt.rcParams['toolbar'] = 'None'
if GENERATE_IMAGES:
    plt.ion()
    # It is intended to be used in a script, and not in a Jupyter, since it does not update the graph but regenerates it. For the moment I'm just saving it.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
if RUNNING_ON_DELFOS:
    alsubaie_dataset_path = '/data/datasets/Alsubaie/Data/'
else:
    alsubaie_dataset_path = '../Alsubaie/Data/'

dataset = WSSBDatasetTest(alsubaie_dataset_path, organ_list=['Colon'], load_at_init=False)
original_image, M_gt = dataset[0]
print('Image shape:', original_image.shape)

In [ ]:
# Generate all images derivated from the ground truth.

img_np = original_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0).astype('uint8')
img_od = rgb2od(original_image)

C_gt = direct_deconvolution(img_od, M_gt)
M_gt = M_gt.unsqueeze(0) # (1, 3, 2)
C_gt = C_gt.unsqueeze(0) # (1, 2, 224, 224)

M_gt_rgb = torch.clamp(od2rgb(M_gt / np.log(256)), 0.0, 255.0) # (1, 3, 2)
M_gt_rgb_np = M_gt_rgb.squeeze().detach().cpu().numpy().transpose(1, 0).astype('int') # (2, 3)

C_H_gt_od = C_gt[:, 0, :, :].squeeze() # (224, 224)
C_H_gt = torch.clamp(od2rgb(C_H_gt_od), 0.0, 255.0)
C_H_gt_np = C_gt[:, 0, :, :].squeeze().numpy() # (224, 224)

C_E_gt_od = C_gt[:, 1, :, :].squeeze() # (224, 224)
C_E_gt = torch.clamp(od2rgb(C_E_gt_od), 0.0, 255.0)
C_E_gt_np = C_gt[:, 1, :, :].squeeze().numpy() # (224, 224)

H_gt_od = torch.einsum('bcs,bshw->bschw', M_gt, C_gt)[:,0,:,:] # (batch_size, H, W)
H_gt = torch.clamp(od2rgb(H_gt_od), 0.0, 255.0) # (batch_size, 3, H, W)
E_gt_od = torch.einsum('bcs,bshw->bschw', M_gt, C_gt)[:,1,:,:] # (batch_size, H, W)
E_gt = torch.clamp(od2rgb(E_gt_od), 0.0, 255.0) # (batch_size, 3, H, W)

H_gt_np = H_gt.squeeze().detach().cpu().numpy().transpose(1, 2, 0).astype('uint8')
E_gt_np = E_gt.squeeze().detach().cpu().numpy().transpose(1, 2, 0).astype('uint8')

In [ ]:
# Plot the ground truth images.
# This could also be generated during training, so it is not strictly necessary to run this cell.

fig, ax = plt.subplots(1, 5, figsize=(20, 4))

ax[0].imshow(img_np)
ax[0].set_title('Original Image')
ax[0].axis('off')

ax[1].imshow(C_H_gt_np, cmap='gray')
ax[1].set_title('Original Hematoxylin\nConcentration')
ax[1].axis('off')

ax[2].imshow(C_E_gt_np, cmap='gray')
ax[2].set_title('Original Eosin\nConcentration')
ax[2].axis('off')

ax[3].imshow(H_gt_np)
ax[3].set_title('Original Hematoxylin')
ax[3].axis('off')

ax[4].imshow(E_gt_np)
ax[4].set_title('Original Eosin')
ax[4].axis('off')

plt.show()

In [ ]:
model = Cnet().to(device)
input_noise = torch.rand(original_image.shape).unsqueeze(0).to(device)  # Unsqueezed to add the batch dimension, it needs to be ([1, 3, x, y])
original_tensor = original_image.unsqueeze(0).to(device)                

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
train_loss = list()

for epoch in range(100000):
    optimizer.zero_grad()

    C_mean = model(input_noise)

    # Generate the colors matrix as a sample of a gaussian distribution given the Ruifrok matrix
    h_matrix, e_matrix = random_ruifrok_matrix_variation(0.05)
    M_matrix = np.concatenate((h_matrix,e_matrix),axis=1)                # ([1, 3, 2])
    M_matrix = torch.from_numpy(M_matrix).float().unsqueeze(0)           # ([1, 2, x, y])

    # Generate the 3 channels image and get it back to RGB
    reconstructed = torch.einsum('bcs,bshw->bchw', M_matrix, C_mean)   # ([1, 3, x, y])
    reconstructed = torch.clamp(od2rgb(reconstructed), 0, 255.0)              

    loss = torch.nn.functional.mse_loss(reconstructed, original_tensor)
    train_loss.append(loss.item())
    loss.backward()
    optimizer.step()

    # Inteded to be used in a script, and not in a Jupyter, since it does not update the graph but regenerates it in the cell output. For the moment I'm just saving it.
    if GENERATE_IMAGES and epoch % 500 == 0:
        # Plot the ground truth images
        fig, ax = plt.subplots(2, 5, figsize=(20, 9))

        ax[0,0].imshow(img_np)
        ax[0,0].set_title('Original Image')
        ax[0,0].axis('off')

        ax[0,1].imshow(C_H_gt_np, cmap='gray')
        ax[0,1].set_title('Original Hematoxylin\nConcentration')
        ax[0,1].axis('off')

        ax[0,2].imshow(C_E_gt_np, cmap='gray')
        ax[0,2].set_title('Original Eosin\nConcentration')
        ax[0,2].axis('off')

        ax[0,3].imshow(H_gt_np)
        ax[0,3].set_title('Original Hematoxylin')
        ax[0,3].axis('off')

        ax[0,4].imshow(E_gt_np)
        ax[0,4].set_title('Original Eosin')
        ax[0,4].axis('off')

        # Generate the images from the model
        C_mean = C_mean.detach().cpu()
        img_rec_np = reconstructed.squeeze().detach().cpu().numpy().transpose(1, 2, 0).astype('uint8')
        
        C_H_rec_od = C_mean[:, 0, :, :].squeeze() 
        C_H_rec = torch.clamp(od2rgb(C_H_rec_od), 0.0, 255.0)
        C_H_rec_np = C_mean[:, 0, :, :].squeeze().numpy() 
        C_E_rec_od = C_mean[:, 1, :, :].squeeze()
        C_E_rec = torch.clamp(od2rgb(C_E_rec_od), 0.0, 255.0)
        C_E_rec_np = C_mean[:, 1, :, :].squeeze().numpy()

        H_rec_od = torch.einsum('bcs,bshw->bschw', M_matrix, C_mean)[:,0,:,:] 
        H_rec = torch.clamp(od2rgb(H_rec_od), 0.0, 255.0) 
        E_rec_od = torch.einsum('bcs,bshw->bschw', M_matrix, C_mean)[:,1,:,:]
        E_rec = torch.clamp(od2rgb(E_rec_od), 0.0, 255.0)

        H_rec_np = H_rec.squeeze().detach().cpu().numpy().transpose(1, 2, 0).astype('uint8')
        E_rec_np = E_rec.squeeze().detach().cpu().numpy().transpose(1, 2, 0).astype('uint8')

        # Plot the generated images via the model
        ax[1,0].imshow(img_rec_np)
        ax[1,0].set_title('Reconstructed Image')
        ax[1,0].axis('off')

        ax[1,1].imshow(C_H_rec_np, cmap='gray')
        ax[1,1].set_title('Reconstructed Hematoxylin\nConcentration')
        ax[1,1].axis('off')

        ax[1,2].imshow(C_E_rec_np, cmap='gray')
        ax[1,2].set_title('Reconstructed Eosin\nConcentration')
        ax[1,2].axis('off')

        ax[1,3].imshow(H_rec_np)
        ax[1,3].set_title('Reconstructed Hematoxylin')
        ax[1,3].axis('off')

        ax[1,4].imshow(E_rec_np)
        ax[1,4].set_title('Reconstructed Eosin')
        ax[1,4].axis('off')
        
        if RUNNING_ON_DELFOS:
            plt.savefig(f'/home/modej/Deep_Var_BCD/results/dip/{epoch}.png', transparent=True)
        else:
            plt.savefig(f'/home/modejota/Deep_Var_BCD/results/dip/{epoch}.png', transparent=True)
